In [ ]:
import os
import sys
import cv2
import numpy as np
import torch
import pickle
from matplotlib import pyplot as plt


## Model

In [ ]:
from omegaconf import OmegaConf
from scripts.inference import load_model_from_config
from ldm.util import instantiate_from_config

In [ ]:
def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    model.cuda()
    model.eval()
    return model

In [ ]:
config = OmegaConf.load("configs/nusc.yaml")
model = load_model_from_config(config, "checkpoints/model.ckpt")

In [ ]:
model